In [240]:
import numpy as np
from pyscf import gto, scf, ao2mo, fci
import CC, Wick
from math import factorial, sqrt
import itertools, copy, collections
from numbers import Number

In [2]:
bohr = 0.529177249

H2sep = 1.605 * bohr

mol = gto.Mole()
mol.verbose = 1
mol.atom = 'H 0 0 0; H 0 0 ' + str(H2sep)
mol.basis = 'sto-3g'
mol.spin = 0
mol.build()

Enuc = mol.energy_nuc()

mf = scf.ROHF(mol)
mf.kernel()

h1 = mf.mo_coeff.T.dot(mf.get_hcore()).dot(mf.mo_coeff)
eri = ao2mo.kernel(mol, mf.mo_coeff)

cisolver = fci.FCI(mol, mf.mo_coeff)

In [3]:
mf.energy_tot()

-1.1026388111917238

In [4]:
cisolver.kernel()

(-1.1284543355083052,
 array([[ 9.90656547e-01,  2.77555756e-17],
        [ 1.91460513e-17, -1.36380375e-01]]))

In [5]:
h1

array([[-1.18985062e+00,  2.60021255e-17],
       [-4.71423569e-17, -5.33749102e-01]])

In [6]:
eri

array([[6.54009511e-01, 8.32667268e-17, 6.45249427e-01],
       [1.04083409e-16, 1.87521981e-01, 5.55111512e-17],
       [6.45249427e-01, 1.38777878e-16, 6.78136184e-01]])

In [7]:
Norbs = mol.nao
Nocc = mf.nelectron_alpha
vacuum = [1 for i in range(Nocc)] + [0 for i in range (Norbs - Nocc)]

In [8]:
integralTensor2body = np.zeros((Norbs, Norbs, Norbs, Norbs))
for p in range(Norbs):
    for q in range(Norbs):
        for r in range(p + 1):
            for s in range(q + 1):
                x = int(p + Norbs * r - 0.5 * r * (r + 1))
                y = int(q + Norbs * s - 0.5 * s * (s + 1))
                print(p, q, r, s)
                print(x, y)
                if p == r and q == s:
                    integralTensor2body[p,q,r,s] = eri[x, y]
#                    hamiltonian2Body = hamiltonian2Body + 0.5 * eri[x, y] * spinFreeDoubleExcitation(p, q, r, s)
                else:
                    integralTensor2body[p,q,r,s] = eri[x, y]
                    integralTensor2body[r,s,p,q] = np.conjugate(eri[x, y])
#                    hamiltonian2Body = hamiltonian2Body + 0.5 * eri[x, y] * spinFreeDoubleExcitation(p, q, r, s) + 0.5 * np.conjugate(eri[x, y]) * spinFreeDoubleExcitation(r, s, p, q)

0 0 0 0
0 0
0 1 0 0
0 1
0 1 0 1
0 2
1 0 0 0
1 0
1 0 1 0
2 0
1 1 0 0
1 1
1 1 0 1
1 2
1 1 1 0
2 1
1 1 1 1
2 2


In [9]:
integralTensor2body

array([[[[6.54009511e-01, 8.32667268e-17],
         [1.04083409e-16, 1.87521981e-01]],

        [[8.32667268e-17, 6.45249427e-01],
         [0.00000000e+00, 5.55111512e-17]]],


       [[[1.04083409e-16, 0.00000000e+00],
         [6.45249427e-01, 1.38777878e-16]],

        [[1.87521981e-01, 5.55111512e-17],
         [1.38777878e-16, 6.78136184e-01]]]])

In [10]:
integralTensor2body[1,0,1,0]

0.6452494265884844

In [11]:
UGgenerators1body = np.ndarray((Norbs, Norbs), dtype=Wick.operatorSum)
for p in range(Norbs):
    for q in range(Norbs):
        UGgenerators1body[p,q] = CC.spinFreeNtupleExcitation((p,), (q,))

In [12]:
UGgenerators2body = np.ndarray((Norbs, Norbs, Norbs, Norbs), dtype=Wick.operatorSum)
for p in range(Norbs):
    for q in range(Norbs):
        for r in range(Norbs):
            for s in range(Norbs):
                UGgenerators2body[p,q,r,s] = CC.spinFreeNtupleExcitation((p, q), (r, s))

In [13]:
print(UGgenerators2body[0,1,1,0])

1.0 * a^{0\alpha}a^{1\alpha}a_{0\alpha}a_{1\alpha}
 + 1.0 * a^{0\alpha}a^{1\beta}a_{0\beta}a_{1\alpha}
 + 1.0 * a^{0\beta}a^{1\alpha}a_{0\alpha}a_{1\beta}
 + 1.0 * a^{0\beta}a^{1\beta}a_{0\beta}a_{1\beta}


In [14]:
class Index:
    '''
    Class for an orbital index
    Orbital types are 'g' (general), 'h' (inactive hole), 'a' (active), and 'p' (particle)
    '''
    def __init__(self, name, orbitalType):
        self.name = name
        self.orbitalType = orbitalType
        self.tuple = (self.name, self.orbitalType)

    def __hash__(self):
        return hash(self.tuple)

    def __eq__(self, other):
        return self.tuple == other

    def __str__(self):
        return self.name
#        return self.orbitalType + str(self.indexNumber)


In [15]:
class Tensor:
    '''
    Class for amplitude tensors of spin-free excitation operators
    '''
    def __init__(self, name, array, lowerIndicesTuple, upperIndicesTuple):
        self.name = name
        self.lowerIndices = lowerIndicesTuple
        self.upperIndices = upperIndicesTuple
        self.excitationRank = len(self.lowerIndices)
        self.array = array

    def setArray(self, array):
        if array.shape == self.array.shape:
            self.array = array
        else:
            print("Array is of wrong shape")

    def getOperator(self):
        operator = Wick.operatorSum([])
        it = np.nditer(self.array, flags=["multi_index"])
        
        for i in it:
            indices = it.multi_index
            creationIndices = indices[:self.excitationRank]
            annihilationIndices = indices[self.excitationRank:]
            operator += i * CC.spinFreeNtupleExcitation(creationIndices, annihilationIndices)
        return operator

    def __str__(self):
        string = self.name + "_{"
        for p in self.lowerIndices:
            string += p.__str__()
        string += "}^{"
        for q in self.upperIndices:
            string += q.__str__()
        string += "}"
        return string

In [16]:
class productOfTensors:
    def __init__(self, listOfTensors):
        self.name = name
        self.listOfTensors = listOfTensors

    def __str__(self):
        string = ""
        for t in self.listOfTensors:
            string += t.__str__()
        return string

In [17]:
p = Index('p','g')
q = Index('q','g')
r = Index('r','g')
s = Index('s','g')

In [18]:
hpq = Tensor('h', h1, (p,), (q,))

In [19]:
print(hpq)

h_{p}^{q}


In [20]:
hpq.setArray(h1)

In [21]:
h1

array([[-1.18985062e+00,  2.60021255e-17],
       [-4.71423569e-17, -5.33749102e-01]])

In [22]:
print(hpq.getOperator())

-1.1898506186070186 * a^{0\alpha}a_{0\alpha}
 + 2.600212552842601e-17 * a^{0\alpha}a_{1\alpha}
 + -1.1898506186070186 * a^{0\beta}a_{0\beta}
 + 2.600212552842601e-17 * a^{0\beta}a_{1\beta}
 + -4.7142356915405136e-17 * a^{1\alpha}a_{0\alpha}
 + -0.5337491016607854 * a^{1\alpha}a_{1\alpha}
 + -4.7142356915405136e-17 * a^{1\beta}a_{0\beta}
 + -0.5337491016607854 * a^{1\beta}a_{1\beta}


In [23]:
gpqrs = Tensor('g', integralTensor2body, (p, q), (r,s))
print(gpqrs)
print(gpqrs.array)

g_{pq}^{rs}
[[[[6.54009511e-01 8.32667268e-17]
   [1.04083409e-16 1.87521981e-01]]

  [[8.32667268e-17 6.45249427e-01]
   [0.00000000e+00 5.55111512e-17]]]


 [[[1.04083409e-16 0.00000000e+00]
   [6.45249427e-01 1.38777878e-16]]

  [[1.87521981e-01 5.55111512e-17]
   [1.38777878e-16 6.78136184e-01]]]]


In [24]:
i = Index('i', 'g')
j = Index('j', 'g')
Aij = Tensor('A',np.arange(4).reshape(2,2),(i,),(j,))
print(Aij.array)

[[0 1]
 [2 3]]


In [25]:
k = Index('k', 'g')
l = Index('l', 'g')
Bkl = Tensor('B',np.arange(4).reshape(2,2),(k,),(l,))
print(Bkl.array)

[[0 1]
 [2 3]]


In [26]:
Aij.lowerIndices

(<__main__.Index at 0x116e934f0>,)

In [27]:
j != Aij.lowerIndices[0]

True

In [28]:
def getAxis(tensor, index):
    for a in range(tensor.excitationRank):
        if tensor.lowerIndices[a] == index:
            return a
        elif tensor.upperIndices[a] == index:
            return tensor.excitationRank + a

In [29]:
getAxis(gpqrs, s)

3

In [30]:
Wick.vacuumExpectationValue(hpq.getOperator() + 0.5 * gpqrs.getOperator(), [1,0]) + Enuc

-1.1026388111917234

In [31]:
def contract(Tensor1, Tensor2, index1, index2):
    axis1 = getAxis(Tensor1, index1)
    axis2 = getAxis(Tensor2, index2)
    newLowerIndicesList = list(Tensor1.lowerIndices)
    newLowerIndicesList.remove(index1)
    newLowerIndicesList = newLowerIndicesList + list(Tensor2.lowerIndices)
    newUpperIndicesList = list(Tensor2.upperIndices)
    newUpperIndicesList.remove(index2)
    newUpperIndicesList = list(Tensor1.upperIndices) + newUpperIndicesList
    name = Tensor1.name + Tensor2.name
    resultArray = np.tensordot(Tensor1.array, Tensor2.array, axes=(axis1, axis2))
    result = Tensor(name, resultArray, newLowerIndicesList, newUpperIndicesList)
    return result

In [32]:
vacuum = np.array([1,0])

In [33]:
Wick.vacuumExpectationValue(gpqrs.getOperator(), vacuum)

1.3080190227122477

In [34]:
2 * np.tensordot(np.tensordot(vacuum, hpq.array, axes=1), vacuum, axes=1)

-2.379701237214037

In [35]:
2 * np.tensordot(np.tensordot(np.tensordot(np.tensordot(vacuum, gpqrs.array, axes=1), vacuum, axes=1), vacuum, axes=1), vacuum, axes=1)

1.3080190227122477

In [36]:
print(Aij)
print(Aij.array)
print(Bkl)
print(Bkl.array)

A_{i}^{j}
[[0 1]
 [2 3]]
B_{k}^{l}
[[0 1]
 [2 3]]


In [37]:
print(contract(Aij, Bkl, i, l))
print(contract(Aij, Bkl, i, l).getOperator())

AB_{k}^{j}
2.0 * a^{0\alpha}a_{0\alpha}
 + 6.0 * a^{0\alpha}a_{1\alpha}
 + 2.0 * a^{0\beta}a_{0\beta}
 + 6.0 * a^{0\beta}a_{1\beta}
 + 3.0 * a^{1\alpha}a_{0\alpha}
 + 11.0 * a^{1\alpha}a_{1\alpha}
 + 3.0 * a^{1\beta}a_{0\beta}
 + 11.0 * a^{1\beta}a_{1\beta}


In [38]:
a = np.arange(9).reshape((3,3))
print(a)

[[0 1 2]
 [3 4 5]
 [6 7 8]]


In [39]:
b = np.arange(9).reshape((3,3))
print(b)

[[0 1 2]
 [3 4 5]
 [6 7 8]]


In [40]:
np.einsum("ij,kl->il", a, b)

array([[ 27,  36,  45],
       [108, 144, 180],
       [189, 252, 315]])

In [41]:
def addNewIndex(indicesList, orbitalType):
    newIndexName = orbitalType + "_{" + str(len(indicesList)) + "}"
    indicesList.append(Index(newIndexName, orbitalType))
    return indicesList[-1]

In [42]:
generalIndices = []
particleIndices = []
holeIndices = []
activeIndices = []

In [43]:
h = Tensor('h', np.zeros((2,2)), (addNewIndex(generalIndices, 'g'),), (addNewIndex(generalIndices, 'g'),))
h.setArray(h1)
print(h)

h_{g_{0}}^{g_{1}}


In [44]:
g = Tensor('g', np.zeros((2,2,2,2)), (generalIndices[0], addNewIndex(generalIndices, 'g')), (generalIndices[1], addNewIndex(generalIndices, 'g')))
g.setArray(integralTensor2body)
print(g)

g_{g_{0}g_{2}}^{g_{1}g_{3}}


In [45]:
t2 = Tensor('t2', np.zeros((1,1,1,1)), ('p', 'p'), ('h', 'h'))
print(t2)

t2_{pp}^{hh}


In [46]:
t2amplitudes = np.arange(16).reshape((2,2,2,2))

In [47]:
t2new = Tensor('t2', (addNewIndex(particleIndices, 'p'), addNewIndex(particleIndices, 'p')), (addNewIndex(holeIndices, 'h'), addNewIndex(holeIndices, 'h')), Norbs)
print(t2new)

TypeError: 'numpy.int64' object is not iterable

In [444]:
t2.setArray(t2amplitudes)
t2new.setArray(t2amplitudes)

AttributeError: 'list' object has no attribute 'shape'

In [445]:
t2amplitudes[0,0,0,0] = 1

In [446]:
print(t2.array)

[<__main__.Index object at 0x11baabe50>, <__main__.Index object at 0x11baab1c0>]


In [677]:
class Tensor:
    '''
    Class for amplitude tensors of spin-free excitation operators
    '''
    def __init__(self, name, lowerIndexTypesList, upperIndexTypesList, array=None):
        self.name = name
        self.lowerIndexTypes = lowerIndexTypesList
        self.upperIndexTypes = upperIndexTypesList
        self.excitationRank = len(self.lowerIndexTypes)
        self.array = np.zeros(self.getShape())
        if array is not None:
            self.setArray(array)

    def getShape(self):
        shapeList = []
        for iType in self.lowerIndexTypes:
            if iType == 'g':
                shapeList.append(Norbs)
            elif iType == 'p':
                shapeList.append(Norbs - Nocc)
            elif iType == 'h':
                shapeList.append(Nocc)
            else:
                print('Orbital index type Error')
        for iType in self.upperIndexTypes:
            if iType == 'g':
                shapeList.append(Norbs)
            elif iType == 'p':
                shapeList.append(Norbs - Nocc)
            elif iType == 'h':
                shapeList.append(Nocc)
            else:
                print('Orbital index type Error')
        return tuple(shapeList)

    def setArray(self, array):
        if array.shape == self.array.shape:
            self.array = array
        else:
            print("Array is of wrong shape")

    def getOperator(self):
        operator = Wick.operatorSum([])
        it = np.nditer(self.array, flags=["multi_index"])
        
        for i in it:
            indices = it.multi_index
            creationIndices = indices[:self.excitationRank]
            annihilationIndices = indices[self.excitationRank:]
            operator += i * CC.spinFreeNtupleExcitation(creationIndices, annihilationIndices)
        return operator

    def getDiagrams(self):
        diagrams = []
        lowerGeneralIndexCount = sum(i == 'g' for i in self.lowerIndexTypes)
        lowerSplits = list(itertools.combinations_with_replacement(['h', 'p'], lowerGeneralIndexCount))
        upperGeneralIndexCount = sum(i == 'g' for i in self.upperIndexTypes)
        upperSplits = list(itertools.combinations_with_replacement(['h', 'p'], upperGeneralIndexCount))
        for lowerSplit in lowerSplits:
            lowerSlices = [slice(None)] * self.excitationRank
            lowerSplitIndexTypes = list(lowerSplit)
            lGI = 0
            newLowerIndexTypes = copy.copy(self.lowerIndexTypes)
            for lI in range(len(newLowerIndexTypes)):
                if newLowerIndexTypes[lI] == 'g':
                    newLI = lowerSplitIndexTypes[lGI]
                    if newLI == 'h':
                        lowerSlices[lI] = slice(None,Nocc)
                    elif newLI == 'p':
                        lowerSlices[lI] = slice(Nocc, None)
                    newLowerIndexTypes[lI] = newLI
                    lGI += 1
            for upperSplit in upperSplits:
                upperSlices = [slice(None)] * self.excitationRank
                upperSplitIndexTypes = list(upperSplit)
                uGI = 0
                newUpperIndexTypes = copy.copy(self.upperIndexTypes)
                for uI in range(len(newUpperIndexTypes)):
                    if newUpperIndexTypes[uI] == 'g':
                        newUI = upperSplitIndexTypes[uGI]
                        if newUI == 'h':
                            upperSlices[uI] = slice(None,Nocc)
                        elif newUI == 'p':
                            upperSlices[uI] = slice(Nocc, None)
                        newUpperIndexTypes[uI] = newUI
                        uGI += 1
                slices = lowerSlices + upperSlices
                print(lowerSplitIndexTypes)
                print(upperSplitIndexTypes)
                print(newLowerIndexTypes)
                print(newUpperIndexTypes)
                print(slices)
                diagram = Tensor(self.name, newLowerIndexTypes, newUpperIndexTypes, self.array[slices])
                diagrams.append(diagram)
        return diagrams

    def __add__(self, other):
        if isinstance(other, Tensor):
            return TensorSum([TensorProduct([self]), TensorProduct([other])])

    def __mul__(self, other):
        if isinstance(other, Tensor):
            return TensorProduct([self, other])
        elif isinstance(other, Number):
            return TensorProduct([self], other)
        else:
            return NotImplemented

    def __rmul__(self, other):
        if isinstance(other, Number):
            return TensorProduct([self], other)
        else:
            return NotImplemented

    def __str__(self):
        string = self.name + "_{"
        for p in self.lowerIndexTypes:
            string += p.__str__()
        string += "}^{"
        for q in self.upperIndexTypes:
            string += q.__str__()
        string += "}"
        return string

In [678]:
class Vertex:
    '''
    Class for amplitude tensors of spin-free excitation operators
    '''
    def __init__(self, tensor, lowerIndicesList, upperIndicesList):
        self.name = tensor.name
        self.tensor = tensor
        self.lowerIndices = lowerIndicesList
        self.upperIndices = upperIndicesList
        self.excitationRank = len(self.lowerIndices)

    def getOperator(self):
        operator = Wick.operatorSum([])
        it = np.nditer(self.tensor.array, flags=["multi_index"])
        
        for i in it:
            indices = it.multi_index
            creationIndices = indices[:self.excitationRank]
            annihilationIndices = indices[self.excitationRank:]
            operator += i * CC.spinFreeNtupleExcitation(creationIndices, annihilationIndices)
        return operator

    def __str__(self):
        string = self.name + "_{"
        for p in self.lowerIndices:
            string += p.__str__()
        string += "}^{"
        for q in self.upperIndices:
            string += q.__str__()
        string += "}"
        return string

In [679]:
f = Tensor('f', ['g'], ['g'], h1)

In [680]:
f.array

array([[-1.18985062e+00,  2.60021255e-17],
       [-4.71423569e-17, -5.33749102e-01]])

In [681]:
f.array.shape

(2, 2)

In [683]:
diagrams = f.getDiagrams()

['h']
['h']
['h']
['h']
[slice(None, 1, None), slice(None, 1, None)]
['h']
['p']
['h']
['p']
[slice(None, 1, None), slice(1, None, None)]
['p']
['h']
['p']
['h']
[slice(1, None, None), slice(None, 1, None)]
['p']
['p']
['p']
['p']
[slice(1, None, None), slice(1, None, None)]


In [685]:
print(diagrams[3].array)

[[-0.5337491]]


In [686]:
print(f)

f_{g}^{g}


In [687]:
v = Tensor('v', ['g', 'g'], ['g', 'g'], integralTensor2body)

In [688]:
v.array

array([[[[6.54009511e-01, 8.32667268e-17],
         [1.04083409e-16, 1.87521981e-01]],

        [[8.32667268e-17, 6.45249427e-01],
         [0.00000000e+00, 5.55111512e-17]]],


       [[[1.04083409e-16, 0.00000000e+00],
         [6.45249427e-01, 1.38777878e-16]],

        [[1.87521981e-01, 5.55111512e-17],
         [1.38777878e-16, 6.78136184e-01]]]])

In [689]:
print(v)

v_{gg}^{gg}


In [690]:
diagrams = v.getDiagrams()

['h', 'h']
['h', 'h']
['h', 'h']
['h', 'h']
[slice(None, 1, None), slice(None, 1, None), slice(None, 1, None), slice(None, 1, None)]
['h', 'h']
['h', 'p']
['h', 'h']
['h', 'p']
[slice(None, 1, None), slice(None, 1, None), slice(None, 1, None), slice(1, None, None)]
['h', 'h']
['p', 'p']
['h', 'h']
['p', 'p']
[slice(None, 1, None), slice(None, 1, None), slice(1, None, None), slice(1, None, None)]
['h', 'p']
['h', 'h']
['h', 'p']
['h', 'h']
[slice(None, 1, None), slice(1, None, None), slice(None, 1, None), slice(None, 1, None)]
['h', 'p']
['h', 'p']
['h', 'p']
['h', 'p']
[slice(None, 1, None), slice(1, None, None), slice(None, 1, None), slice(1, None, None)]
['h', 'p']
['p', 'p']
['h', 'p']
['p', 'p']
[slice(None, 1, None), slice(1, None, None), slice(1, None, None), slice(1, None, None)]
['p', 'p']
['h', 'h']
['p', 'p']
['h', 'h']
[slice(1, None, None), slice(1, None, None), slice(None, 1, None), slice(None, 1, None)]
['p', 'p']
['h', 'p']
['p', 'p']
['h', 'p']
[slice(1, None, None), sl

In [477]:
Norbs

2

In [478]:
t2 = Tensor('t', ['p', 'p'], ['h', 'h'])

In [479]:
t2.array

array([[[[0.]]]])

In [480]:
generalIndices = []
particleIndices = []
holeIndices = []
activeIndices = []

In [481]:
fock = Vertex(f, [addNewIndex(generalIndices, 'g')], [addNewIndex(generalIndices, 'g')])

In [482]:
print(fock)

f_{g_{0}}^{g_{1}}


In [483]:
fock.lowerIndices

In [484]:
for i in fock.lowerIndices:
    print(i)

g_{0}


In [485]:
print(generalIndices[0])

g_{0}


In [486]:
fock.lowerIndices[0] = generalIndices[0]

In [639]:
class TensorProduct:
    def __init__(self, tensorList, prefactor=1.):
        self.tensorList = tensorList
        self.lowerIndices = {'g':[], 'p':[], 'h':[], 'a':[]}
        self.upperIndices = {'g':[], 'p':[], 'h':[], 'a':[]}
        self.vertexList = self.getVertexList(tensorList)
        self.prefactor = prefactor

    def addNewIndex(self, orbitalType, lowerBool):
        count = len(self.lowerIndices[orbitalType]) + len(self.upperIndices[orbitalType])
        newIndexName = orbitalType + "_{" + str(count) + "}"
        newIndex = Index(newIndexName, orbitalType)
        if lowerBool:
            self.lowerIndices[orbitalType].append(newIndex)
        else:
            self.upperIndices[orbitalType].append(newIndex)
        return newIndex

    def getVertexList(self, tensorList_):
        vertexList = []
        for t in tensorList_:
            lowerIndexList = []
            for i in t.lowerIndexTypes:
                lowerIndexList.append(self.addNewIndex(i, True))
            upperIndexList = []
            for i in t.upperIndexTypes:
                upperIndexList.append(self.addNewIndex(i, False))
            vertexList.append(Vertex(t, lowerIndexList, upperIndexList))
        return vertexList

    def contractibleIndices(self):
        contractiblePairsList = []
        for v1 in range(len(self.vertexList) - 1):
            possibleUpperIndices = [i1 for i1 in self.vertexList[v1].upperIndices if (i1.orbitalType == 'p' or i1.orbitalType == 'g')]
            for v2 in range(v1 + 1, len(self.vertexList)):
                possibleLowerIndices = [i2 for i2 in self.vertexList[v2].lowerIndices if (i2.orbitalType == 'p' or i2.orbitalType == 'g')]
                contractiblePairsList += list(itertools.product(possibleLowerIndices, possibleUpperIndices))
        for v1 in range(len(self.vertexList) - 1):
            possibleLowerIndices = [i1 for i1 in self.vertexList[v1].lowerIndices if (i1.orbitalType == 'h' or i1.orbitalType == 'g')]
            for v2 in range(v1 + 1, len(self.vertexList)):
                possibleUpperIndices = [i2 for i2 in self.vertexList[v2].upperIndices if (i2.orbitalType == 'h' or i2.orbitalType == 'g')]
                contractiblePairsList += list(itertools.product(possibleLowerIndices, possibleUpperIndices))
        return contractiblePairsList

    def getFullyContractedTopologies(self):
        contractiblePairs = self.contractibleIndices()
        totalNumberOfIndices = sum([len(l) for l in self.lowerIndices.values()]) + sum([len(l) for l in self.upperIndices.values()])
        possibleTopologies = itertools.combinations(contractiblePairs, int(totalNumberOfIndices / 2))
        return [topology for topology in possibleTopologies if len(set(itertools.chain(*topology))) == totalNumberOfIndices]

    def getTopologiesLeavingFreeIndices(self, freeLowerIndexTypeList, freeUpperIndexTypeList, allowGeneral=True):
        currentFreeLowerIndexCounts = {}
        currentFreeUpperIndexCounts = {}
        freeLowerIndexCounts = {}
        freeUpperIndexCounts = {}
        for iType in self.upperIndices:
            currentFreeUpperIndexCounts[iType] = len(self.upperIndices[iType])
            currentFreeLowerIndexCounts[iType] = len(self.lowerIndices[iType])
            freeLowerIndexCounts[iType] = 0
            freeUpperIndexCounts[iType] = 0
        for iType in freeLowerIndexTypeList:
            freeLowerIndexCounts[iType] += 1
        for iType in freeUpperIndexTypeList:
            freeUpperIndexCounts[iType] += 1
        contractiblePairs = self.contractibleIndices()
#        currentFreeLowerIndexTypeList = []
#        for iType in self.lowerIndices:
#            currentFreeLowerIndexTypeList += [iType] * len(self.lowerIndices[iType])
#        currentFreeUpperIndexTypeList = []
#        for iType in self.upperIndices:
#            currentFreeUpperIndexTypeList += [iType] * len(self.upperIndices[iType])
        numberOfContractions = sum([len(l) for l in self.lowerIndices.values()]) - len(freeLowerIndexTypeList)
        possibleTopologies = itertools.combinations(contractiblePairs, numberOfContractions)
        chosenTopologies = []
        for topology in possibleTopologies:
            contractedLowerIndexCounts = {'g':0, 'p':0, 'h':0, 'a':0}
            contractedUpperIndexCounts = {'g':0, 'p':0, 'h':0, 'a':0}
            for pair in topology:
                contractedLowerIndexCounts[pair[0].orbitalType] += 1
                contractedUpperIndexCounts[pair[1].orbitalType] += 1
                print(pair[0], pair[1])
#            contractedLowerIndexTypes = [pair[0].orbitalType for pair in topology]
#            contractedUpperIndexTypes = [pair[1].orbitalType for pair in topology]
#            if collections.Counter(freeLowerIndexTypeList + contractedLowerIndexTypes) == collections.Counter(currentFreeLowerIndexTypeList) and collections.Counter(freeUpperIndexTypeList + contractedUpperIndexTypes) == collections.Counter(currentFreeUpperIndexTypeList):
            topologyAllowed = False
            if allowGeneral and len(set(itertools.chain(*topology))) == 2 * numberOfContractions:
                spareGeneralLowerIndexCount = currentFreeLowerIndexCounts['g'] - contractedLowerIndexCounts['g']
                spareGeneralUpperIndexCount = currentFreeUpperIndexCounts['g'] - contractedUpperIndexCounts['g']
                print(spareGeneralLowerIndexCount)
                for lgp in range(spareGeneralLowerIndexCount + 1):
                    for lgh in range(spareGeneralLowerIndexCount + 1 - lgp):
                        for ugp in range(spareGeneralUpperIndexCount + 1):
                            for ugh in range(spareGeneralUpperIndexCount + 1 - ugp):
                                print(currentFreeLowerIndexCounts['p'] - contractedLowerIndexCounts['p'] + lgp, freeLowerIndexCounts['p'], currentFreeLowerIndexCounts['h'] - contractedLowerIndexCounts['h'] + lgh, freeLowerIndexCounts['h'], currentFreeUpperIndexCounts['p'] - contractedUpperIndexCounts['p'] + ugp, freeUpperIndexCounts['p'], currentFreeUpperIndexCounts['h'] - contractedUpperIndexCounts['h'] + ugh, freeUpperIndexCounts['h'])
                                if (currentFreeLowerIndexCounts['p'] - contractedLowerIndexCounts['p'] + lgp == freeLowerIndexCounts['p']) and (currentFreeLowerIndexCounts['h'] - contractedLowerIndexCounts['h'] + lgh == freeLowerIndexCounts['h']) and (currentFreeUpperIndexCounts['p'] - contractedUpperIndexCounts['p'] + ugp == freeUpperIndexCounts['p']) and (currentFreeUpperIndexCounts['h'] - contractedUpperIndexCounts['h'] + ugh == freeUpperIndexCounts['h']):
                                    topologyAllowed = True
            elif len(set(itertools.chain(*topology))) == 2 * numberOfContractions:
                if all([(currentFreeLowerIndexCounts[itype] - contractedLowerIndexCounts[iType] == freeLowerIndexCounts[iType]) for itype in self.lowerIndices]) and all([(currentFreeUpperIndexCounts[iType] - contractedUpperIndexCounts[iType] == freeUpperIndexCounts[itype]) for itype in self.upperIndices]):
                    topologyAllowed = True
            if topologyAllowed:
                chosenTopologies.append(topology)
        return chosenTopologies

    def __mul__(self, other):
        if isinstance(other, Tensor):
            return TensorProduct(self.tensorList + [other])
        elif isinstance(other, TensorProduct):
            return TensorProduct(self.tensorList + other.tensorList)
        elif isinstance(other, Number):
            return TensorProduct(self.tensorList, self.prefactor * other)

    def __rmul__(self, other):
        if isinstance(other, Tensor):
            return TensorProduct([other] + self.tensorList)
        elif isinstance(other, Number):
            return TensorProduct(self.tensorList, other * self.prefactor)

    def __str__(self):
        string = ''
        for v in self.vertexList:
            string += v.__str__()
        return string

In [640]:
tP = v * t2

In [641]:
print(tP)

v_{g_{0}g_{1}}^{g_{2}g_{3}}t_{p_{0}p_{1}}^{h_{0}h_{1}}


In [642]:
sum([len(l) for l in tP.lowerIndices.values()]) - len(['h', 'h'])

2

In [692]:
len(tP.getTopologiesLeavingFreeIndices(['p', 'p'], ['h', 'h']))

p_{0} g_{2}
p_{0} g_{3}
p_{0} g_{2}
p_{1} g_{2}
p_{0} g_{2}
p_{1} g_{3}
2
0 2 0 0 0 0 2 2
0 2 1 0 0 0 2 2
0 2 2 0 0 0 2 2
1 2 0 0 0 0 2 2
1 2 1 0 0 0 2 2
2 2 0 0 0 0 2 2
p_{0} g_{2}
g_{0} h_{0}
1
1 2 0 0 0 0 1 2
1 2 0 0 0 0 2 2
1 2 0 0 1 0 1 2
1 2 1 0 0 0 1 2
1 2 1 0 0 0 2 2
1 2 1 0 1 0 1 2
2 2 0 0 0 0 1 2
2 2 0 0 0 0 2 2
2 2 0 0 1 0 1 2
p_{0} g_{2}
g_{0} h_{1}
1
1 2 0 0 0 0 1 2
1 2 0 0 0 0 2 2
1 2 0 0 1 0 1 2
1 2 1 0 0 0 1 2
1 2 1 0 0 0 2 2
1 2 1 0 1 0 1 2
2 2 0 0 0 0 1 2
2 2 0 0 0 0 2 2
2 2 0 0 1 0 1 2
p_{0} g_{2}
g_{1} h_{0}
1
1 2 0 0 0 0 1 2
1 2 0 0 0 0 2 2
1 2 0 0 1 0 1 2
1 2 1 0 0 0 1 2
1 2 1 0 0 0 2 2
1 2 1 0 1 0 1 2
2 2 0 0 0 0 1 2
2 2 0 0 0 0 2 2
2 2 0 0 1 0 1 2
p_{0} g_{2}
g_{1} h_{1}
1
1 2 0 0 0 0 1 2
1 2 0 0 0 0 2 2
1 2 0 0 1 0 1 2
1 2 1 0 0 0 1 2
1 2 1 0 0 0 2 2
1 2 1 0 1 0 1 2
2 2 0 0 0 0 1 2
2 2 0 0 0 0 2 2
2 2 0 0 1 0 1 2
p_{0} g_{3}
p_{1} g_{2}
2
0 2 0 0 0 0 2 2
0 2 1 0 0 0 2 2
0 2 2 0 0 0 2 2
1 2 0 0 0 0 2 2
1 2 1 0 0 0 2 2
2 2 0 0 0 0 2 2
p_{0} g_{3}
p_{1} g_{3}
p_{0

20

In [644]:
doublesResidualTopologies = tP.getTopologiesLeavingFreeIndices(['p', 'p'], ['h', 'h'])

p_{0} g_{2}
p_{0} g_{3}
p_{0} g_{2}
p_{1} g_{2}
p_{0} g_{2}
p_{1} g_{3}
2
0 2 0 0 0 0 2 2
0 2 1 0 0 0 2 2
0 2 2 0 0 0 2 2
1 2 0 0 0 0 2 2
1 2 1 0 0 0 2 2
2 2 0 0 0 0 2 2
p_{0} g_{2}
g_{0} h_{0}
1
1 2 0 0 0 0 1 2
1 2 0 0 0 0 2 2
1 2 0 0 1 0 1 2
1 2 1 0 0 0 1 2
1 2 1 0 0 0 2 2
1 2 1 0 1 0 1 2
2 2 0 0 0 0 1 2
2 2 0 0 0 0 2 2
2 2 0 0 1 0 1 2
p_{0} g_{2}
g_{0} h_{1}
1
1 2 0 0 0 0 1 2
1 2 0 0 0 0 2 2
1 2 0 0 1 0 1 2
1 2 1 0 0 0 1 2
1 2 1 0 0 0 2 2
1 2 1 0 1 0 1 2
2 2 0 0 0 0 1 2
2 2 0 0 0 0 2 2
2 2 0 0 1 0 1 2
p_{0} g_{2}
g_{1} h_{0}
1
1 2 0 0 0 0 1 2
1 2 0 0 0 0 2 2
1 2 0 0 1 0 1 2
1 2 1 0 0 0 1 2
1 2 1 0 0 0 2 2
1 2 1 0 1 0 1 2
2 2 0 0 0 0 1 2
2 2 0 0 0 0 2 2
2 2 0 0 1 0 1 2
p_{0} g_{2}
g_{1} h_{1}
1
1 2 0 0 0 0 1 2
1 2 0 0 0 0 2 2
1 2 0 0 1 0 1 2
1 2 1 0 0 0 1 2
1 2 1 0 0 0 2 2
1 2 1 0 1 0 1 2
2 2 0 0 0 0 1 2
2 2 0 0 0 0 2 2
2 2 0 0 1 0 1 2
p_{0} g_{3}
p_{1} g_{2}
2
0 2 0 0 0 0 2 2
0 2 1 0 0 0 2 2
0 2 2 0 0 0 2 2
1 2 0 0 0 0 2 2
1 2 1 0 0 0 2 2
2 2 0 0 0 0 2 2
p_{0} g_{3}
p_{1} g_{3}
p_{0

In [645]:
len(doublesResidualTopologies)

20

In [651]:
for pair in doublesResidualTopologies[5]:
    print(pair[0], pair[1])

p_{0} g_{3}
p_{1} g_{2}


In [584]:
for pair in tP.contractibleIndices():
    print(pair[0], pair[1])

p_{0} g_{2}
p_{0} g_{3}
p_{1} g_{2}
p_{1} g_{3}
g_{0} h_{0}
g_{0} h_{1}
g_{1} h_{0}
g_{1} h_{1}


In [490]:
tP.lowerIndices

{'g': [<__main__.Index at 0x11721a4c0>, <__main__.Index at 0x11721ad00>],
 'p': [<__main__.Index at 0x117fa1c10>, <__main__.Index at 0x117fa1af0>],
 'h': [],
 'a': []}

In [491]:
for iT in tP.lowerIndices:
    print(iT)
    print(tP.lowerIndices[iT])

g
[<__main__.Index object at 0x11721a4c0>, <__main__.Index object at 0x11721ad00>]
p
[<__main__.Index object at 0x117fa1c10>, <__main__.Index object at 0x117fa1af0>]
h
[]
a
[]


In [694]:
term = TensorProduct([v,t2])

In [695]:
print(term)

v_{g_{0}g_{1}}^{g_{2}g_{3}}t_{p_{0}p_{1}}^{h_{0}h_{1}}


In [494]:
print(term.vertexList[0].lowerIndices)

[<__main__.Index object at 0x1171ec2e0>, <__main__.Index object at 0x1171ec2b0>]


In [696]:
contractiblePairs = term.contractibleIndices()

In [496]:
for p in contractiblePairs:
    print(p[0], p[1])

p_{0} g_{2}
p_{0} g_{3}
p_{1} g_{2}
p_{1} g_{3}
g_{0} h_{0}
g_{0} h_{1}
g_{1} h_{0}
g_{1} h_{1}


In [693]:
class ContractedTerm:
    def __init__(self, tensorProduct, contractionTopology):
        self.tensorList = tensorProduct.tensorList
        self.prefactor = tensorProduct.prefactor
        self.lowerIndices = copy.deepcopy(tensorProduct.lowerIndices)
        self.upperIndices = copy.deepcopy(tensorProduct.upperIndices)
        self.vertexList = copy.deepcopy(tensorProduct.vertexList)
        self.contractionTopology = contractionTopology
        self.contractedIndicesList = []
        self.applyContractions(self.contractionTopology)
        self.loops = self.getLoops()
        self.prefactor = self.prefactor * pow(-2, len(self.loops))
        self.prefactor = self.prefactor * pow(-1, sum((i.orbitalType == 'h' or i.orbitalType == 'a') for i in self.contractedIndicesList))

    def contractibleIndices(self):
        contractiblePairsList = []
        for v1 in range(len(self.vertexList) - 1):
            possibleFirstIndices = [i1 for i1 in self.vertexList[v1].upperIndices if (i1.orbitalType == 'p' or i1.orbitalType == 'g')]
            for v2 in range(v1 + 1, len(self.vertexList)):
                possibleSecondIndices = [i2 for i2 in self.vertexList[v2].lowerIndices if (i2.orbitalType == 'p' or i2.orbitalType == 'g')]
                contractiblePairsList += list(itertools.product(possibleFirstIndices, possibleSecondIndices))
        for v1 in range(len(self.vertexList) - 1):
            possibleFirstIndices = [i1 for i1 in self.vertexList[v1].lowerIndices if (i1.orbitalType == 'h' or i1.orbitalType == 'g')]
            for v2 in range(v1 + 1, len(self.vertexList)):
                possibleSecondIndices = [i2 for i2 in self.vertexList[v2].upperIndices if (i2.orbitalType == 'h' or i2.orbitalType == 'g')]
                contractiblePairsList += list(itertools.product(possibleFirstIndices, possibleSecondIndices))
        return contractiblePairsList

    def contractIndexPair(self, indexPair):
        for vertex in self.vertexList:
            keep = 0
            discard = 1
            if indexPair[0].orbitalType == 'a':
                pass
            elif indexPair[1].orbitalType == 'a':
                keep = 1
                discard = 0
            elif indexPair[0].orbitalType == 'h' or indexPair[0].orbitalType == 'p':
                pass
            elif indexPair[1].orbitalType == 'h' or indexPair[1].orbitalType == 'p':
                keep = 1
                discard = 0
            else:
                pass
            if indexPair[keep] not in self.contractedIndicesList:
                vertex.upperIndices = [indexPair[keep] if i.name == indexPair[discard].name else i for i in vertex.upperIndices]
                vertex.lowerIndices = [indexPair[keep] if i.name == indexPair[discard].name else i for i in vertex.lowerIndices]
                self.contractedIndicesList.append(indexPair[keep])
#            vertex = self.vertexList[v]
#            for i in range(len(vertex.lowerIndices)):
#                if vertex.upperIndices[i] is indexPair[1]:
#                    self.vertexList[v].upperIndices[i] = indexPair[0]
#            for i in range(len(vertex.upperIndices)):
#                if vertex.lowerIndices[i] is indexPair[1]:
#                    self.vertexList[v].lowerIndices[i] = indexPair[0]
        for iType in self.lowerIndices:
            for index in self.lowerIndices[iType]:
                if index.name == indexPair[0].name:
                    self.lowerIndices[iType].remove(index)
        for iType in self.upperIndices:
            for index in self.upperIndices[iType]:
                if index.name == indexPair[1].name:
                    self.upperIndices[iType].remove(index)

    def applyContractions(self, topology):
        for indexPair in topology:
            self.contractIndexPair(indexPair)

    def followIndexOut(self, ingoingIndex):
        found = False
        outgoingIndex = ingoingIndex
        for vertex in self.vertexList:
            for i in range(len(vertex.upperIndices)):
                if vertex.upperIndices[i] == ingoingIndex and not found:
                    outgoingIndex = vertex.lowerIndices[i]
                    found = True
                elif vertex.upperIndices[i] == ingoingIndex:
                    print("Error: index appears more than once")
        return outgoingIndex

    def getLoops(self):
        loopsList = []
        pool = copy.copy(self.contractedIndicesList)
        while len(pool) > 0:
            loopList = []
            start = pool[0]
            contraction = 0
            while True:
                inIndex = pool[contraction]
                outIndex = self.followIndexOut(inIndex)
                loopList.append(outIndex)
                pool.remove(pool[contraction])
                if outIndex == start:
                    break
                for p in range(len(pool)):
                    if pool[p] == outIndex:
                        contraction = p
            loopsList.append(tuple(loopList))
        return tuple(loopsList)

    def __add__(self, other):
        if isinstance(other, ContractedTerm):
            return TensorSum([self, other])

    def __radd__(self, other):
        if other == 0:
            return self


    def __str__(self):
        string = str(self.prefactor) + ' * '
        for vertex in self.vertexList:
            string += vertex.__str__()
        return string

In [697]:
fullyContractedTopologies = term.getFullyContractedTopologies()

In [698]:
contractedTerm = ContractedTerm(term, fullyContractedTopologies[0])

In [500]:
for pair in fullyContractedTopologies[0]:
    print(pair[0], pair[1])

p_{0} g_{2}
p_{1} g_{3}
g_{0} h_{0}
g_{1} h_{1}


In [501]:
print(term)

v_{g_{0}g_{1}}^{g_{2}g_{3}}t_{p_{0}p_{1}}^{h_{0}h_{1}}


In [699]:
print(contractedTerm)

4.0 * v_{h_{0}h_{1}}^{p_{0}p_{1}}t_{p_{0}p_{1}}^{h_{0}h_{1}}


In [703]:
contractedTerm.vertexList[0].lowerIndices

[<__main__.Index at 0x117b73910>, <__main__.Index at 0x117b73280>]

In [503]:
contractedTerm.contractIndexPair(fullyContractedTopologies[0][0])

In [504]:
print(contractedTerm)

4.0 * v_{h_{0}h_{1}}^{p_{0}p_{1}}t_{p_{0}p_{1}}^{h_{0}h_{1}}


In [505]:
contractedTerms = [ContractedTerm(term, topology) for topology in fullyContractedTopologies]

In [506]:
for cTerm in contractedTerms:
    print(cTerm)

4.0 * v_{h_{0}h_{1}}^{p_{0}p_{1}}t_{p_{0}p_{1}}^{h_{0}h_{1}}
-2.0 * v_{h_{1}h_{0}}^{p_{0}p_{1}}t_{p_{0}p_{1}}^{h_{0}h_{1}}
-2.0 * v_{h_{0}h_{1}}^{p_{1}p_{0}}t_{p_{0}p_{1}}^{h_{0}h_{1}}
4.0 * v_{h_{1}h_{0}}^{p_{1}p_{0}}t_{p_{0}p_{1}}^{h_{0}h_{1}}


In [507]:
contractedTerm.lowerIndices

{'g': [], 'p': [], 'h': [], 'a': []}

In [508]:
term.vertexList[0].lowerIndices

[<__main__.Index at 0x1171ec2e0>, <__main__.Index at 0x1171ec2b0>]

In [509]:
topologies = []
topologies.append([])
topologies[0].append(contractiblePairs[0])
possiblePairsList = contractiblePairs[1:]

In [510]:
for p in possiblePairsList:
    print(p[0] in topologies[0][0] or p[1] in topologies[0][0])

True
True
False
False
False
False
False


In [511]:
topologies.append([])
topologies[1].append(contractiblePairs[1])
possiblePairsList = contractiblePairs[2:]

In [512]:
for p in possiblePairsList:
    print(p[0] in topologies[1][0] or p[1] in topologies[1][0])

False
True
False
False
False
False


In [513]:
def getContractionTopologies(contractiblePairsList):
    possiblePairsList = contractiblePairsList
    

In [514]:
contractionTopology = []

In [515]:
i = 0
while i < len(contractiblePairs):
    print(i)
    alreadyContracted = False
    j = 0
    while j < len(contractionTopology):
        c = contractionTopology[j]
        if p[0] in c or p[1] in c:
            alreadyContracted = True
            j += 1
        else:
            j += 1
    if not alreadyContracted:
        contractionTopology.append(contractiblePairs[i])
    i += 1

0
1
2
3
4
5
6
7


In [516]:
print(contractionTopology)
print(contractionTopology[0][0], contractionTopology[0][1])

[(<__main__.Index object at 0x11850c7f0>, <__main__.Index object at 0x1171ecc70>), (<__main__.Index object at 0x11850c7f0>, <__main__.Index object at 0x1171ecac0>), (<__main__.Index object at 0x11850c340>, <__main__.Index object at 0x1171ecc70>), (<__main__.Index object at 0x11850c340>, <__main__.Index object at 0x1171ecac0>), (<__main__.Index object at 0x1171ec2e0>, <__main__.Index object at 0x117237d90>), (<__main__.Index object at 0x1171ec2e0>, <__main__.Index object at 0x117237220>)]
p_{0} g_{2}


In [517]:
print(v)

v_{gg}^{gg}


In [518]:
print(t2)

t_{pp}^{hh}


In [519]:
transformFluctuation = TensorProduct([v, t2])

In [520]:
print(transformFluctuation)

v_{g_{0}g_{1}}^{g_{2}g_{3}}t_{p_{0}p_{1}}^{h_{0}h_{1}}


In [521]:
fullyContractedTransformFluctuation = transformFluctuation.getFullyContractedTopologies()

In [522]:
print(fullyContractedTransformFluctuation)

[((<__main__.Index object at 0x118554a30>, <__main__.Index object at 0x1185326d0>), (<__main__.Index object at 0x118554ca0>, <__main__.Index object at 0x1185321c0>), (<__main__.Index object at 0x118532670>, <__main__.Index object at 0x1185543a0>), (<__main__.Index object at 0x118532e20>, <__main__.Index object at 0x118554310>)), ((<__main__.Index object at 0x118554a30>, <__main__.Index object at 0x1185326d0>), (<__main__.Index object at 0x118554ca0>, <__main__.Index object at 0x1185321c0>), (<__main__.Index object at 0x118532670>, <__main__.Index object at 0x118554310>), (<__main__.Index object at 0x118532e20>, <__main__.Index object at 0x1185543a0>)), ((<__main__.Index object at 0x118554a30>, <__main__.Index object at 0x1185321c0>), (<__main__.Index object at 0x118554ca0>, <__main__.Index object at 0x1185326d0>), (<__main__.Index object at 0x118532670>, <__main__.Index object at 0x1185543a0>), (<__main__.Index object at 0x118532e20>, <__main__.Index object at 0x118554310>)), ((<__main

In [523]:
energyTerm = TensorProduct([v, t2], 0.5)
CCDEnergyTerms = [ContractedTerm(energyTerm, topology) for topology in energyTerm.getFullyContractedTopologies()]

In [524]:
print(energyTerm)

v_{g_{0}g_{1}}^{g_{2}g_{3}}t_{p_{0}p_{1}}^{h_{0}h_{1}}


In [525]:
print(CCDEnergyTerms)

[<__main__.ContractedTerm object at 0x1189a15e0>, <__main__.ContractedTerm object at 0x117b820a0>, <__main__.ContractedTerm object at 0x117b7a0d0>, <__main__.ContractedTerm object at 0x117b7aa00>]


In [526]:
cTerm0 = CCDEnergyTerms[0]
print(cTerm0)

2.0 * v_{h_{0}h_{1}}^{p_{0}p_{1}}t_{p_{0}p_{1}}^{h_{0}h_{1}}


In [527]:
testIndex = cTerm0.vertexList[0].upperIndices[1]

In [528]:
print(testIndex)

p_{1}


In [529]:
def followIndexOut(term, ingoingIndex):
    found = False
    outgoingIndex = ingoingIndex
    for vertex in term.vertexList:
        for i in range(len(vertex.upperIndices)):
            if vertex.upperIndices[i] == ingoingIndex and not found:
                outgoingIndex = vertex.lowerIndices[i]
                found = True
            elif vertex.upperIndices[i] == ingoingIndex:
                print("Error: index appears more than once")
    return outgoingIndex

In [530]:
testedIndex = cTerm0.followIndexOut(testIndex)

In [531]:
print(testedIndex)

h_{1}


In [532]:
print(cTerm0.vertexList[0].lowerIndices[0])

h_{0}


In [533]:
print(cTerm0.contractionTopology)

((<__main__.Index object at 0x11893ea00>, <__main__.Index object at 0x116ebb1f0>), (<__main__.Index object at 0x11893e2e0>, <__main__.Index object at 0x1186ead60>), (<__main__.Index object at 0x116ebb8b0>, <__main__.Index object at 0x11893e790>), (<__main__.Index object at 0x116ebb7c0>, <__main__.Index object at 0x11893ee50>))


In [534]:
list(cTerm0.contractionTopology)

[(<__main__.Index at 0x11893ea00>, <__main__.Index at 0x116ebb1f0>),
 (<__main__.Index at 0x11893e2e0>, <__main__.Index at 0x1186ead60>),
 (<__main__.Index at 0x116ebb8b0>, <__main__.Index at 0x11893e790>),
 (<__main__.Index at 0x116ebb7c0>, <__main__.Index at 0x11893ee50>)]

In [535]:
for i in cTerm0.contractedIndicesList:
    print(i)

p_{0}
p_{1}
h_{0}
h_{1}


In [536]:
for p in cTerm0.contractionTopology:
    print(p[0], p[1])

p_{0} g_{2}
p_{1} g_{3}
g_{0} h_{0}
g_{1} h_{1}


In [537]:
def getLoops(contractedTerm):
    loopsList = []
    pool = copy.copy(contractedTerm.contractedIndicesList)
    while len(pool) > 0:
#    for i in range(10):
        loopList = []
        start = pool[0]
#        print(start)
        contraction = 0
        while True:
#        for i in range(10):
            inIndex = pool[contraction]
#            print(inIndex)
            outIndex = contractedTerm.followIndexOut(inIndex)
#            print(contraction)
#            print(outIndex)
            loopList.append(outIndex)
#            print(pool)
            pool.remove(pool[contraction])
#            print(pool)
            if outIndex == start:
                break
            for p in range(len(pool)):
                if pool[p] == outIndex:
                    contraction = p
        loopsList.append(tuple(loopList))
    return tuple(loopsList)

In [538]:
print(getLoops(CCDEnergyTerms[3]))

((<__main__.Index object at 0x11893e790>, <__main__.Index object at 0x11850cac0>), (<__main__.Index object at 0x11893ee50>, <__main__.Index object at 0x116e54d60>))


In [539]:
CCDEnergyEquation = sum(CCDEnergyTerms)

In [540]:
print(CCDEnergyEquation)

None


In [541]:
for cTerm in CCDEnergyTerms:
    print(cTerm)

2.0 * v_{h_{0}h_{1}}^{p_{0}p_{1}}t_{p_{0}p_{1}}^{h_{0}h_{1}}
-1.0 * v_{h_{1}h_{0}}^{p_{0}p_{1}}t_{p_{0}p_{1}}^{h_{0}h_{1}}
-1.0 * v_{h_{0}h_{1}}^{p_{1}p_{0}}t_{p_{0}p_{1}}^{h_{0}h_{1}}
2.0 * v_{h_{1}h_{0}}^{p_{1}p_{0}}t_{p_{0}p_{1}}^{h_{0}h_{1}}


In [542]:
projectedSingleExcitationTensors = []
for h in range(Nocc):
    for p in range(Nocc, Norbs):
        name = "\Psi_{" + str(p) + "}^{" + str(h) + "}"
        projectedSingleExcitationTensors.append(Tensor(name, ['p'], ['h']))
        projectedSingleExcitationTensors[-1].array[p-Nocc, h] = 1

In [543]:
projectedSingleExcitationTensors[0].name

'\\Psi_{1}^{0}'

In [544]:
projectedDoubleExcitationTensors = []
for h in range(Nocc):
    for h2 in range(Nocc):
        for p1 in range(Nocc, Norbs):
            for p2 in range(Nocc, Norbs):
                name = "\Psi_{" + str(p1) + str(p2) + "}^{" + str(h) + str(h2) + "}"
                projectedDoubleExcitationTensors.append(Tensor(name, ['p', 'p'], ['h', 'h']))
                projectedDoubleExcitationTensors[-1].array[p1-Nocc, p2-Nocc, h, h2] = 1

In [545]:
projectedDoubleExcitationTensors[0].name

'\\Psi_{11}^{00}'

In [546]:
class TensorSum:
    def __init__(self, summandList):
        self.summandList = summandList

    def __add__(self, other):
        if isinstance(other, TensorSum):
            return TensorSum(self.summandList + other.summandList)

    def __mul__(self, other):
        if isinstance(other, Tensor) or isinstance(other, TensorProduct) or isinstance(other, TensorSum) or isinstance(other, Number):
            return TensorSum([summand * other for summand in self.summandList])

    def __rmul__(self, other):
        if isinstance(other, Tensor) or isinstance(other, TensorProduct) or isinstance(other, TensorSum) or isinstance(other, Number):
            return TensorSum([other * summand for summand in self.summandList])

    def __str__(self):
        if len(self.summandList) == 0:
            return ""
        string = self.summandList[0].__str__()
        for summand in self.summandList[1:]:
            string += "\n + "
            string += summand.__str__()
        return string

In [547]:
print(TensorSum(CCDEnergyTerms))

2.0 * v_{h_{0}h_{1}}^{p_{0}p_{1}}t_{p_{0}p_{1}}^{h_{0}h_{1}}
 + -1.0 * v_{h_{1}h_{0}}^{p_{0}p_{1}}t_{p_{0}p_{1}}^{h_{0}h_{1}}
 + -1.0 * v_{h_{0}h_{1}}^{p_{1}p_{0}}t_{p_{0}p_{1}}^{h_{0}h_{1}}
 + 2.0 * v_{h_{1}h_{0}}^{p_{1}p_{0}}t_{p_{0}p_{1}}^{h_{0}h_{1}}


In [548]:
v * t2

In [549]:
t2 * v

In [550]:
projectedSingleExcitationTensor = Tensor("\Phi", ['p'], ['h'], np.ones((Norbs - Nocc, Nocc)))

In [551]:
projectedDoubleExcitationTensor = Tensor("\Phi", ['h', 'h'], ['p', 'p'], np.ones((Nocc, Nocc, Norbs - Nocc, Norbs - Nocc)))

In [552]:
amplitude = projectedDoubleExcitationTensor * v * t2

In [553]:
print(amplitude)

\Phi_{h_{0}h_{1}}^{p_{0}p_{1}}v_{g_{0}g_{1}}^{g_{2}g_{3}}t_{p_{2}p_{3}}^{h_{2}h_{3}}


In [554]:
for top in amplitude.getFullyContractedTopologies():
    for pair in top:
        print(pair[0], pair[1])
    print("___")

g_{0} p_{0}
g_{1} p_{1}
p_{2} g_{2}
p_{3} g_{3}
h_{0} h_{2}
h_{1} h_{3}
___
g_{0} p_{0}
g_{1} p_{1}
p_{2} g_{2}
p_{3} g_{3}
h_{0} h_{3}
h_{1} h_{2}
___
g_{0} p_{0}
g_{1} p_{1}
p_{2} g_{3}
p_{3} g_{2}
h_{0} h_{2}
h_{1} h_{3}
___
g_{0} p_{0}
g_{1} p_{1}
p_{2} g_{3}
p_{3} g_{2}
h_{0} h_{3}
h_{1} h_{2}
___
g_{0} p_{0}
p_{2} p_{1}
p_{3} g_{2}
h_{0} g_{3}
h_{1} h_{2}
g_{1} h_{3}
___
g_{0} p_{0}
p_{2} p_{1}
p_{3} g_{2}
h_{0} g_{3}
h_{1} h_{3}
g_{1} h_{2}
___
g_{0} p_{0}
p_{2} p_{1}
p_{3} g_{2}
h_{1} g_{3}
h_{0} h_{2}
g_{1} h_{3}
___
g_{0} p_{0}
p_{2} p_{1}
p_{3} g_{2}
h_{1} g_{3}
h_{0} h_{3}
g_{1} h_{2}
___
g_{0} p_{0}
p_{2} p_{1}
p_{3} g_{3}
h_{0} g_{2}
h_{1} h_{2}
g_{1} h_{3}
___
g_{0} p_{0}
p_{2} p_{1}
p_{3} g_{3}
h_{0} g_{2}
h_{1} h_{3}
g_{1} h_{2}
___
g_{0} p_{0}
p_{2} p_{1}
p_{3} g_{3}
h_{1} g_{2}
h_{0} h_{2}
g_{1} h_{3}
___
g_{0} p_{0}
p_{2} p_{1}
p_{3} g_{3}
h_{1} g_{2}
h_{0} h_{3}
g_{1} h_{2}
___
g_{0} p_{0}
p_{3} p_{1}
p_{2} g_{2}
h_{0} g_{3}
h_{1} h_{2}
g_{1} h_{3}
___
g_{0} p_{0}


In [555]:
hamiltonianTensorSum = f + v

In [556]:
print(hamiltonianTensorSum)

f_{g_{0}}^{g_{1}}
 + v_{g_{0}g_{1}}^{g_{2}g_{3}}


In [557]:
v * t2

In [558]:
for tensor in hamiltonianTensorSum.summandList:
    energy = tensor * t2
    for top in energy.getFullyContractedTopologies():
        print(ContractedTerm(energy, top))

4.0 * v_{h_{0}h_{1}}^{p_{0}p_{1}}t_{p_{0}p_{1}}^{h_{0}h_{1}}
-2.0 * v_{h_{1}h_{0}}^{p_{0}p_{1}}t_{p_{0}p_{1}}^{h_{0}h_{1}}
-2.0 * v_{h_{0}h_{1}}^{p_{1}p_{0}}t_{p_{0}p_{1}}^{h_{0}h_{1}}
4.0 * v_{h_{1}h_{0}}^{p_{1}p_{0}}t_{p_{0}p_{1}}^{h_{0}h_{1}}


In [559]:
for tensor in hamiltonianTensorSum.summandList:
    partiallyContracted = tensor * t2
    for top in partiallyContracted.getTopologiesLeavingFreeIndices(['p', 'p'], ['h', 'h']):
        print(ContractedTerm(partiallyContracted, top))

In [560]:
projectedDoubleExcitationTensor.lowerIndexTypes

['h', 'h']

In [561]:
def commutator(operator1, operator2):
    return operator1 * operator2 + (-1) * (operator2 * operator1)

In [562]:
def BCHSimilarityTransform(H, T, order):
    result = H
    for k in range(order):
        nestedCommutator = H
        for i in range(k + 1):
            nestedCommutator = commutator(nestedCommutator, T)
        result += (1 / factorial(k + 1)) * nestedCommutator
    return result

In [563]:
CCDEnergyEquation = BCHSimilarityTransform(hamiltonianTensorSum, t2, 4)

In [564]:
t2 * hamiltonianTensorSum

In [565]:
t2

In [566]:
print(t2 * hamiltonianTensorSum.summandList[1])

t_{p_{0}p_{1}}^{h_{0}h_{1}}v_{g_{0}g_{1}}^{g_{2}g_{3}}


In [567]:
t2

In [568]:
print(CCDEnergyEquation)

f_{g_{0}}^{g_{1}}
 + v_{g_{0}g_{1}}^{g_{2}g_{3}}
 + f_{g_{0}}^{g_{1}}t_{p_{0}p_{1}}^{h_{0}h_{1}}
 + v_{g_{0}g_{1}}^{g_{2}g_{3}}t_{p_{0}p_{1}}^{h_{0}h_{1}}
 + t_{p_{0}p_{1}}^{h_{0}h_{1}}f_{g_{0}}^{g_{1}}
 + t_{p_{0}p_{1}}^{h_{0}h_{1}}v_{g_{0}g_{1}}^{g_{2}g_{3}}
 + f_{g_{0}}^{g_{1}}t_{p_{0}p_{1}}^{h_{0}h_{1}}t_{p_{2}p_{3}}^{h_{2}h_{3}}
 + v_{g_{0}g_{1}}^{g_{2}g_{3}}t_{p_{0}p_{1}}^{h_{0}h_{1}}t_{p_{2}p_{3}}^{h_{2}h_{3}}
 + t_{p_{0}p_{1}}^{h_{0}h_{1}}f_{g_{0}}^{g_{1}}t_{p_{2}p_{3}}^{h_{2}h_{3}}
 + t_{p_{0}p_{1}}^{h_{0}h_{1}}v_{g_{0}g_{1}}^{g_{2}g_{3}}t_{p_{2}p_{3}}^{h_{2}h_{3}}
 + t_{p_{0}p_{1}}^{h_{0}h_{1}}f_{g_{0}}^{g_{1}}t_{p_{2}p_{3}}^{h_{2}h_{3}}
 + t_{p_{0}p_{1}}^{h_{0}h_{1}}v_{g_{0}g_{1}}^{g_{2}g_{3}}t_{p_{2}p_{3}}^{h_{2}h_{3}}
 + t_{p_{0}p_{1}}^{h_{0}h_{1}}t_{p_{2}p_{3}}^{h_{2}h_{3}}f_{g_{0}}^{g_{1}}
 + t_{p_{0}p_{1}}^{h_{0}h_{1}}t_{p_{2}p_{3}}^{h_{2}h_{3}}v_{g_{0}g_{1}}^{g_{2}g_{3}}
 + f_{g_{0}}^{g_{1}}t_{p_{0}p_{1}}^{h_{0}h_{1}}t_{p_{2}p_{3}}^{h_{2}h_{3}}t_{p_{4}p_{5}}^{h_{4}h_{5

In [570]:
len(CCDEnergyEquation.summandList)

62

In [575]:
i = 0
for term in CCDEnergyEquation.summandList:
    j = 0
    print(i)
    for top in term.getFullyContractedTopologies():
        print(j)
        print(ContractedTerm(term, top))
        j =+ 1
    i += 1

0
1
2
3
0
4.0 * v_{h_{0}h_{1}}^{p_{0}p_{1}}t_{p_{0}p_{1}}^{h_{0}h_{1}}
1
-2.0 * v_{h_{1}h_{0}}^{p_{0}p_{1}}t_{p_{0}p_{1}}^{h_{0}h_{1}}
1
-2.0 * v_{h_{0}h_{1}}^{p_{1}p_{0}}t_{p_{0}p_{1}}^{h_{0}h_{1}}
1
4.0 * v_{h_{1}h_{0}}^{p_{1}p_{0}}t_{p_{0}p_{1}}^{h_{0}h_{1}}
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
